<a href="https://colab.research.google.com/github/Arnavdas/Market-data-extraction-sentimental-analysis/blob/master/Market_data_extraction_sentimental_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import nltk
import os
os.chdir('C:/Users/LENOVO/Desktop')
from lxml import html
import requests
from bs4 import BeautifulSoup
import re
import textstat
import pandas as pd
import numpy as np
# reads the links and positive-negative words
df = pd.read_excel(r'C:\Users\LENOVO\Desktop\Internshala\LoughranMcDonald_MasterDictionary_2018.xlsx') #for an earlier version of Excel, you may need to use the file extension of 'xls'
df1 = pd.read_excel(r'C:\Users\LENOVO\Desktop\Internshala\cik_list - Copy.xlsx') #for an earlier version of Excel, you may need to use the file extension of 'xls'
#print (df)

In [0]:
# SYLLABLE SEARCHING Function
from nltk.corpus import cmudict
dk = cmudict.dict()
def nsyl(word):
    #print([list(y for y in x if y[-1].isdigit()) for x in dk[word.lower()]] )
    return [len(list(y for y in x if y[-1].isdigit())) for x in dk[word.lower()]]

In [0]:
# READING ALL TEXT FILES OF STOPWORDS AND SETTING UP A DICT FOR THEM
stopw = {}; save = {}
os.chdir("C:/Users/LENOVO/Desktop/Internshala")
for file in os.listdir("C:/Users/LENOVO/Desktop/Internshala"):
    if file.endswith(".txt"):
        #print(str(file))
        if str(file) != 'output.txt':
            file1 = open(str(file),"r")
            for x in file1:
                stopw[x.split()[0]] = 0
        else:
            file2 = open(str(file),"r")
            for x in file2:
                save[x.split()[0]] = []

In [0]:
# SETTING UP DICT FOR + AND - WORDS
dff = df.to_numpy(); dff_dic = {}
for i in range(len(dff)):
    #kk = np.where(df==i.upper())[0][0]
    dff_dic[dff[i][0]] = [df.iloc()[i]['Negative'], df.iloc()[i]['Positive'], df.iloc()[i]['Uncertainty'],  df.iloc()[i]['Constraining']]
#dff_dic  

In [0]:
import numpy as np

def saver(save, k, neg=np.nan, pos=np.nan, cln=np.nan, syl=np.nan, unc=np.nan, const=np.nan, sents=np.nan):
    if k == 1:
        qt = 'mda'
    elif k == 3:
        qt = 'qqdmr'
    elif k == 2:
        qt = 'rf'
    asl = cln/sents
    pcw = syl/cln
    fi = 0.4*(asl + pcw)
    save[qt+'_positive_score'].append(pos)#1
    save[qt+'_positive_word_proportion'].append(pos/cln)#2
    save[qt+'_negative_score'].append(neg)#3
    save[qt+'_negative_word_proportion'].append(neg/cln)#4
    save[qt+'_polarity_score'].append((pos - neg)/(pos + neg + 0.000001))#5 
    save[qt+'_average_sentence_length'].append(asl)#6
    save[qt+'_complex_word_count'].append(syl)#7
    save[qt+'_constraining_word_proportion'].append(const/cln)#8
    save[qt+'_percentage_of_complex_words'].append(pcw)#9
    save[qt+'_word_count'].append(cln)#10
    save[qt+'_fog_index'].append(fi)#11
    save[qt+'_uncertainty_word_proportion'].append(unc/cln)#12
    save[qt+'_uncertainty_score'].append(unc)#13
    save[qt+'_constraining_score'].append(const)#14

In [0]:
# SENTIMENTAL ANALYSIS, POSITIVITY SOCRE, NEGATIVITY SCORE, POLARITY
def analysis_part(par, stopw, dff_dic, save):
    #print(par)
    tok = {}; const_count = 0
    for jj in range(1, 4):
        if len(par[jj]) != 0:
            for i in par[jj]:
                neg = 0; pos = 0; cln = 0; syl = 0; unc = 0; const = 0
                for j in nltk.word_tokenize(i):
                    if j.isalpha():# chekcs for only letters
                        try :# checks if the word is a stopword or not
                            if stopw[j.upper()]== 0 : # is a stopword
                                pass
                        except:# isn't a stopword
                            try:# checks if a word is atleast 2-syllable
                                if nsyl(j)[0] > 2:
                                    syl+=1
                            except:
                                pass
                            try:#checks for pos, neg, unc, const scores
                                cln+=1
                                if dff_dic[j.upper()][0] > 0:
                                    neg+=1
                                elif dff_dic[j.upper()][1] > 0:
                                    pos+=1
                                unc+= dff_dic[j.upper()][2]
                                const+= dff_dic[j.upper()][3]
                                if dff_dic[j.upper()][3] > 0:
                                    const_count+=1
                            except:
                                pass
            if cln == 0:
                saver(save, jj)
            else:
                sents = textstat.sentence_count(" ".join(par[jj]))
                saver(save, jj, neg, pos, cln, syl, unc, const, sents)        
        else:
            saver(save, jj)
    save['constraining_words_whole_report'].append(const_count)
            
            

In [0]:
atm = re.compile(".*Management's Discussions* and Analysis.*", re.IGNORECASE)
atm1 = re.compile(".*Risk Factors*.*", re.IGNORECASE)
atm2 = re.compile(".*Quantitative and Qualitative Disclosures* about Markets* Risks*.*", re.IGNORECASE)
mat = re.compile("item{1} \d{1}.*\.{1}", re.IGNORECASE )
part = re.compile("PART +[I*V*]+")
tam = re.compile("<page>", re.IGNORECASE)
table = re.compile("<\W*TABLE>"); cntr = 0

for wez in df1['Links'].to_numpy():
    cntr+=1
    wez = ''.join(wez.strip().split())
    pg = requests.get(wez)
    sup = BeautifulSoup(pg.text, 'html.parser')
    par = {1:[], 2:[], 3:[]}; nb = 0# not broken
    if 'page' in [tag.name for tag in sup.find_all()]:
        for i in range(len(sup.page.contents)):# searches only contents in pages tag
            fr = nltk.tokenize.line_tokenize(str(sup.page.contents[i])); y = len(fr)
            for j in range(y):# searches amongst all new lines in page tags only
                if mat.match(fr[j]) != None or nb != 0 :# searches for 'item /d.' in fitst place only
                    if atm.search(fr[j]) != None or nb == 1:# Management's Discussion and Analysis
                        nb = 0
                        for k in range(j+1, y):# inner loop to find only this section specific
                            if mat.match(fr[k]) != None or part.search(fr[k]) != None:# searches untill there's no next ITEM X.
                                break
                            elif k == y-1:
                                nb = 1
                            else:
                                par[1].append(fr[k])
                    elif atm1.search(fr[j]) != None or nb == 2:# Risk factors
                        nb = 0
                        for k in range(j+1, y):
                            if mat.match(fr[k]) != None or part.search(fr[k]) != None:
                                break
                            elif k == y-1:
                                nb = 2
                            else:
                                par[2].append(fr[k])
                    elif atm2.search(fr[j]) != None or nb == 3:# Quantitative and Qualitative Disclosures about Market Risk   
                        nb = 0
                        for k in range(j+1, y):
                            if mat.match(fr[k]) != None or part.search(fr[k]) != None:
                                break
                            elif k == y-1:
                                nb = 3
                            else:
                                par[3].append(fr[k])
                if nb != 0 :
                    break
        #print('cntr:',cntr)
        analysis_part(par, stopw, dff_dic, save)
    else:
        #print('cntr:',cntr)
        save['constraining_words_whole_report'].append(np.nan)
        for i in range(1, 4):
            saver(save, i)

cntr: 1
cntr: 2
cntr: 3
cntr: 4
cntr: 5
cntr: 6
cntr: 7
cntr: 8
cntr: 9
cntr: 10
cntr: 11
cntr: 12
cntr: 13
cntr: 14
cntr: 15
cntr: 16
cntr: 17
cntr: 18
cntr: 19
cntr: 20
cntr: 21
cntr: 22
cntr: 23
cntr: 24
cntr: 25
cntr: 26
cntr: 27
cntr: 28
cntr: 29
cntr: 30
cntr: 31
cntr: 32
cntr: 33
cntr: 34
cntr: 35
cntr: 36
cntr: 37
cntr: 38
cntr: 39
cntr: 40
cntr: 41
cntr: 42
cntr: 43
cntr: 44
cntr: 45
cntr: 46
cntr: 47
cntr: 48
cntr: 49
cntr: 50
cntr: 51
cntr: 52
cntr: 53
cntr: 54
cntr: 55
cntr: 56
cntr: 57
cntr: 58
cntr: 59
cntr: 60
cntr: 61
cntr: 62
cntr: 63
cntr: 64
cntr: 65
cntr: 66
cntr: 67
cntr: 68
cntr: 69
cntr: 70
cntr: 71
cntr: 72
cntr: 73
cntr: 74
cntr: 75
cntr: 76
cntr: 77
cntr: 78
cntr: 79
cntr: 80
cntr: 81
cntr: 82
cntr: 83
cntr: 84
cntr: 85
cntr: 86
cntr: 87
cntr: 88
cntr: 89
cntr: 90
cntr: 91
cntr: 92
cntr: 93
cntr: 94
cntr: 95
cntr: 96
cntr: 97
cntr: 98
cntr: 99
cntr: 100
cntr: 101
cntr: 102
cntr: 103
cntr: 104
cntr: 105
cntr: 106
cntr: 107
cntr: 108
cntr: 109
cntr: 110
cntr: 11

In [0]:
set([len(save[i]) for i in save.keys()])
    

{152}

In [0]:
for i in save.keys():
    df1[i] = save[i]
df1.columns

Index(['CIK', 'CONAME', 'FYRMO', 'FDATE', 'FORM', 'Links', 'Unnamed: 6',
       'mda_positive_score', 'mda_negative_score', 'mda_polarity_score',
       'mda_average_sentence_length', 'mda_percentage_of_complex_words',
       'mda_fog_index', 'mda_complex_word_count', 'mda_word_count',
       'mda_uncertainty_score', 'mda_constraining_score',
       'mda_positive_word_proportion', 'mda_negative_word_proportion',
       'mda_uncertainty_word_proportion', 'mda_constraining_word_proportion',
       'qqdmr_positive_score', 'qqdmr_negative_score', 'qqdmr_polarity_score',
       'qqdmr_average_sentence_length', 'qqdmr_percentage_of_complex_words',
       'qqdmr_fog_index', 'qqdmr_complex_word_count', 'qqdmr_word_count',
       'qqdmr_uncertainty_score', 'qqdmr_constraining_score',
       'qqdmr_positive_word_proportion', 'qqdmr_negative_word_proportion',
       'qqdmr_uncertainty_word_proportion',
       'qqdmr_constraining_word_proportion', 'rf_positive_score',
       'rf_negative_score', '

In [0]:
df1.to_excel('output.xlsx')